# 03 - Feature Engineering
## Year Prediction from Audio Features

### Objectives
1. Create new features from existing ones
2. Feature selection using various methods
3. Dimensionality reduction (PCA)
4. Analyze feature importance

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
import joblib
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Load Processed Data

In [2]:
X_train = np.load('data/splits/X_train.npy')
X_val = np.load('data/splits/X_val.npy')
X_test = np.load('data/splits/X_test.npy')

y_train = np.load('data/splits/y_train.npy')
y_val = np.load('data/splits/y_val.npy')
y_test = np.load('data/splits/y_test.npy')

with open('data/processed/feature_names.txt', 'r') as f:
    feature_names = [line.strip() for line in f.readlines()]

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")
print(f"Features: {len(feature_names)}")

Training set: (394074, 90)
Validation set: (69543, 90)
Test set: (51514, 90)
Features: 90


## 2. Feature Importance Analysis

In [3]:
f_selector = SelectKBest(score_func=f_regression, k='all')
f_selector.fit(X_train, y_train)

f_scores = pd.DataFrame({
    'feature': feature_names,
    'f_score': f_selector.scores_,
    'p_value': f_selector.pvalues_
}).sort_values('f_score', ascending=False)

print("Top 20 Features by F-Score:")
print(f_scores.head(20))

Top 20 Features by F-Score:
          feature       f_score  p_value
0    timbre_avg_1  20996.729500      0.0
5    timbre_avg_6  14569.949411      0.0
2    timbre_avg_3   7867.332874      0.0
62  timbre_cov_51   7100.719207      0.0
39  timbre_cov_28   7016.866278      0.0
6    timbre_avg_7   4917.757638      0.0
68  timbre_cov_57   4839.652670      0.0
45  timbre_cov_34   4802.077115      0.0
56  timbre_cov_45   4631.718814      0.0
35  timbre_cov_24   4571.852337      0.0
46  timbre_cov_35   4477.918642      0.0
66  timbre_cov_55   4433.008116      0.0
58  timbre_cov_47   4071.110326      0.0
32  timbre_cov_21   3893.919377      0.0
11  timbre_avg_12   3859.665145      0.0
72  timbre_cov_61   3746.446537      0.0
77  timbre_cov_66   3297.510402      0.0
19   timbre_cov_8   3200.097444      0.0
67  timbre_cov_56   2901.496021      0.0
73  timbre_cov_62   2833.086820      0.0


In [4]:
fig = px.bar(
    f_scores.head(30),
    x='f_score', y='feature',
    orientation='h',
    title='Top 30 Features by F-Score',
    labels={'f_score': 'F-Score', 'feature': 'Feature'}
)
fig.update_layout(template='plotly_white', height=700, yaxis={'categoryorder': 'total ascending'})
fig.write_html('reports/figures/14_f_score_importance.html')
fig.show()

In [5]:
sample_size = min(50000, len(X_train))
sample_idx = np.random.choice(len(X_train), sample_size, replace=False)

mi_scores = mutual_info_regression(X_train[sample_idx], y_train[sample_idx], random_state=42)

mi_df = pd.DataFrame({
    'feature': feature_names,
    'mi_score': mi_scores
}).sort_values('mi_score', ascending=False)

print("\nTop 20 Features by Mutual Information:")
print(mi_df.head(20))


Top 20 Features by Mutual Information:
          feature  mi_score
0    timbre_avg_1  0.041714
45  timbre_cov_34  0.025192
62  timbre_cov_51  0.023761
5    timbre_avg_6  0.021722
13   timbre_cov_2  0.021644
46  timbre_cov_35  0.018314
39  timbre_cov_28  0.016840
1    timbre_avg_2  0.016778
19   timbre_cov_8  0.016172
70  timbre_cov_59  0.015206
21  timbre_cov_10  0.014411
35  timbre_cov_24  0.014100
6    timbre_avg_7  0.014075
40  timbre_cov_29  0.013155
2    timbre_avg_3  0.013035
3    timbre_avg_4  0.012804
36  timbre_cov_25  0.012719
66  timbre_cov_55  0.012338
73  timbre_cov_62  0.012323
24  timbre_cov_13  0.010900


In [6]:
fig = px.bar(
    mi_df.head(30),
    x='mi_score', y='feature',
    orientation='h',
    title='Top 30 Features by Mutual Information',
    labels={'mi_score': 'Mutual Information', 'feature': 'Feature'}
)
fig.update_layout(template='plotly_white', height=700, yaxis={'categoryorder': 'total ascending'})
fig.write_html('reports/figures/15_mi_importance.html')
fig.show()

In [7]:
print("Training Random Forest for feature importance (using sample)...")
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train[sample_idx], y_train[sample_idx])

rf_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 20 Features by Random Forest Importance:")
print(rf_importance.head(20))

Training Random Forest for feature importance (using sample)...

Top 20 Features by Random Forest Importance:
          feature  importance
0    timbre_avg_1    0.194833
2    timbre_avg_3    0.099632
13   timbre_cov_2    0.080598
1    timbre_avg_2    0.055715
5    timbre_avg_6    0.037630
19   timbre_cov_8    0.026344
56  timbre_cov_45    0.020884
40  timbre_cov_29    0.012985
35  timbre_cov_24    0.012256
6    timbre_avg_7    0.011424
77  timbre_cov_66    0.011153
4    timbre_avg_5    0.010687
62  timbre_cov_51    0.010658
38  timbre_cov_27    0.010550
39  timbre_cov_28    0.010493
3    timbre_avg_4    0.009821
37  timbre_cov_26    0.009790
11  timbre_avg_12    0.009716
15   timbre_cov_4    0.009509
8    timbre_avg_9    0.009137


In [8]:
fig = px.bar(
    rf_importance.head(30),
    x='importance', y='feature',
    orientation='h',
    title='Top 30 Features by Random Forest Importance',
    labels={'importance': 'Importance', 'feature': 'Feature'}
)
fig.update_layout(template='plotly_white', height=700, yaxis={'categoryorder': 'total ascending'})
fig.write_html('reports/figures/16_rf_importance.html')
fig.show()

## 3. Principal Component Analysis (PCA)

In [9]:
pca_full = PCA()
pca_full.fit(X_train)

explained_variance = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print(f"Variance explained by first 10 components: {cumulative_variance[9]*100:.2f}%")
print(f"Variance explained by first 20 components: {cumulative_variance[19]*100:.2f}%")
print(f"Variance explained by first 50 components: {cumulative_variance[49]*100:.2f}%")

Variance explained by first 10 components: 44.72%
Variance explained by first 20 components: 60.70%
Variance explained by first 50 components: 87.23%


In [10]:
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
n_components_99 = np.argmax(cumulative_variance >= 0.99) + 1

print(f"Components needed for 95% variance: {n_components_95}")
print(f"Components needed for 99% variance: {n_components_99}")

Components needed for 95% variance: 67
Components needed for 99% variance: 82


In [11]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=list(range(1, 51)),
    y=explained_variance[:50],
    name='Individual'
))

fig.add_trace(go.Scatter(
    x=list(range(1, 51)),
    y=cumulative_variance[:50],
    name='Cumulative',
    mode='lines+markers'
))

fig.add_hline(y=0.95, line_dash='dash', line_color='red', annotation_text='95% variance')

fig.update_layout(
    title='PCA Explained Variance (First 50 Components)',
    xaxis_title='Principal Component',
    yaxis_title='Explained Variance Ratio',
    template='plotly_white'
)
fig.write_html('reports/figures/17_pca_variance.html')
fig.show()

In [12]:
n_pca_components = 50
pca = PCA(n_components=n_pca_components)

X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

print(f"PCA transformed shapes:")
print(f"  Train: {X_train_pca.shape}")
print(f"  Val: {X_val_pca.shape}")
print(f"  Test: {X_test_pca.shape}")

PCA transformed shapes:
  Train: (394074, 50)
  Val: (69543, 50)
  Test: (51514, 50)


In [13]:
sample_size = 10000
sample_idx = np.random.choice(len(X_train_pca), sample_size, replace=False)

fig = px.scatter(
    x=X_train_pca[sample_idx, 0],
    y=X_train_pca[sample_idx, 1],
    color=y_train[sample_idx],
    title='PCA Components 1 vs 2 (colored by year)',
    labels={'x': 'PC1', 'y': 'PC2', 'color': 'Year'},
    color_continuous_scale='Viridis'
)
fig.update_layout(template='plotly_white')
fig.write_html('reports/figures/18_pca_scatter.html')
fig.show()

## 4. Feature Selection - Top K Features

In [14]:
importance_combined = f_scores.merge(mi_df, on='feature').merge(rf_importance, on='feature')

from sklearn.preprocessing import MinMaxScaler

scaler_norm = MinMaxScaler()
importance_combined['f_score_norm'] = scaler_norm.fit_transform(importance_combined[['f_score']])
importance_combined['mi_score_norm'] = scaler_norm.fit_transform(importance_combined[['mi_score']])
importance_combined['importance_norm'] = scaler_norm.fit_transform(importance_combined[['importance']])

importance_combined['combined_score'] = (
    importance_combined['f_score_norm'] + 
    importance_combined['mi_score_norm'] + 
    importance_combined['importance_norm']
) / 3

importance_combined = importance_combined.sort_values('combined_score', ascending=False)

print("Top 20 Features by Combined Importance:")
print(importance_combined[['feature', 'combined_score']].head(20))

Top 20 Features by Combined Importance:
          feature  combined_score
0    timbre_avg_1        1.000000
1    timbre_avg_6        0.464669
2    timbre_avg_3        0.396730
21   timbre_cov_2        0.349725
3   timbre_cov_51        0.315445
7   timbre_cov_34        0.278168
4   timbre_cov_28        0.258527
66   timbre_avg_2        0.228439
10  timbre_cov_35        0.227187
17   timbre_cov_8        0.220175
5    timbre_avg_7        0.204728
9   timbre_cov_24        0.200881
11  timbre_cov_55        0.175449
8   timbre_cov_45        0.157713
14  timbre_avg_12        0.157191
19  timbre_cov_62        0.153642
28  timbre_cov_29        0.153333
15  timbre_cov_61        0.150142
6   timbre_cov_57        0.148440
41  timbre_cov_10        0.132758


In [15]:
K = 50
top_features = importance_combined['feature'].head(K).tolist()
top_feature_indices = [feature_names.index(f) for f in top_features]

X_train_selected = X_train[:, top_feature_indices]
X_val_selected = X_val[:, top_feature_indices]
X_test_selected = X_test[:, top_feature_indices]

print(f"Selected top {K} features")
print(f"Selected feature shapes: Train {X_train_selected.shape}, Val {X_val_selected.shape}, Test {X_test_selected.shape}")

Selected top 50 features
Selected feature shapes: Train (394074, 50), Val (69543, 50), Test (51514, 50)


In [16]:
fig = px.bar(
    importance_combined.head(30),
    x='combined_score', y='feature',
    orientation='h',
    title='Top 30 Features by Combined Importance Score',
    labels={'combined_score': 'Combined Score', 'feature': 'Feature'}
)
fig.update_layout(template='plotly_white', height=700, yaxis={'categoryorder': 'total ascending'})
fig.write_html('reports/figures/19_combined_importance.html')
fig.show()

## 5. Save Engineered Features

In [17]:
np.save('data/splits/X_train_pca.npy', X_train_pca)
np.save('data/splits/X_val_pca.npy', X_val_pca)
np.save('data/splits/X_test_pca.npy', X_test_pca)

print("PCA transformed data saved")

PCA transformed data saved


In [18]:
np.save('data/splits/X_train_selected.npy', X_train_selected)
np.save('data/splits/X_val_selected.npy', X_val_selected)
np.save('data/splits/X_test_selected.npy', X_test_selected)

with open('data/processed/selected_feature_names.txt', 'w') as f:
    for name in top_features:
        f.write(f"{name}\n")

print("Selected features data saved")

Selected features data saved


In [19]:
joblib.dump(pca, 'models/ml/pca_model.joblib')
print("PCA model saved")

PCA model saved


In [20]:
importance_combined.to_csv('reports/metrics/03_feature_importance.csv', index=False)
print("Feature importance analysis saved")

Feature importance analysis saved
